<h2>Unión de todas las bases </h2>
<p> En este apartado se tratan todas las bases para generar la unión de todas las bases</p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
import pyarrow.feather as feather
pd.pandas.set_option('display.max_columns', None)

In [2]:
# Se carga el archivo tratado del FT004 que fue grabado como Cartera Prestador
FT004 = pd.read_csv('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/carteraprestador_2020_ene_mayo_2021.csv', encoding='utf-8-sig', dtype={'nit_eps':object, 'nitproveedor': object, 'ano': object, 'mes': object})


In [3]:
# Para eliminar todos los espacios 
FT004['nit_eps'] = FT004['nit_eps'].str.replace(" ", "")
FT004['nitproveedor'] = FT004['nitproveedor'].str.replace(" ", "")

In [4]:
FT004 = FT004.groupby(['nit_eps', 'nitproveedor', 'ano', 'mes',]).agg({'CXP':'sum', 'DCXPmes':'sum'}).reset_index()

In [5]:
# Se carga el archivo tratado del FT003 que fue grabado como cxcprestador
FT003 = pd.read_csv('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cxcprestador_2020_ene_may_2021.csv', encoding='utf-8-sig', dtype={'nit_eps':object, 'nitproveedor': object, 'ano': object, 'mes': object})

In [6]:
# Para eliminar todos los espacios 
FT003['nit_eps'] = FT003['nit_eps'].str.replace(" ", "")
FT003['nitproveedor'] = FT003['nitproveedor'].str.replace(" ", "")

In [7]:
FT003 = FT003.groupby(['nit_eps', 'nitproveedor', 'ano', 'mes',]).agg({'CXC':'sum', 'DCXCmes':'sum'}).reset_index()

In [8]:
# Se unen las dos bases de datos para obtener la información de FT004 y FT003 en una sola base de datos
FT004_FT003 = pd.merge(FT004, FT003, how='outer', left_on=['nit_eps', 'nitproveedor', 'ano', 'mes'], right_on=['nit_eps', 'nitproveedor', 'ano', 'mes'], indicator=True)

In [9]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_FT003.rename(columns={'_merge': 'FT004_FT003'}, inplace=True)

In [11]:
# Se carga el archivo tratado del FT005 que fue grabado como causaprestador
FT005 = pd.read_csv('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/CausaPrestador_2020_2021_V2.csv', encoding='utf-8-sig', dtype={'nit_eps':object, 'nitproveedor': object, 'noID': object, 'departamento': object, 'ano': object, 'mes': object})

In [12]:
FT005.columns

Index(['nit_eps', 'nitproveedor', 'ano', 'mes', 'pagomes', 'causmes', 'giro',
       'no_upc', 'compra_cartera', 'pw'],
      dtype='object')

In [13]:
FT005 = FT005.groupby(['nit_eps', 'nitproveedor', 'ano', 'mes',]).agg({'pagomes':'sum', 'causmes':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'pw':'sum'}).reset_index()

In [14]:
# Se unen las dos bases de datos para obtener la información de FT004 y FT003 en una sola base de datos
FT004_FT003_FT005 = pd.merge(FT004_FT003, FT005, how='outer', left_on=['nit_eps', 'nitproveedor', 'ano', 'mes'], right_on=['nit_eps', 'nitproveedor', 'ano', 'mes'], indicator=True)

In [15]:
# Se eliminan los NIT EPS nulos
FT004_FT003_FT005.dropna(subset=['nit_eps'], inplace=True)

In [16]:
del [FT003, FT004, FT004_FT003, FT005]

NameError: name 'FT004_V2' is not defined

<p> La línea siguiente solo se debe ejecutar si se quiere desagregar los prestadores, pues se debe contar con la base de datos del sharepoint EAPB en formato .xlsx. Se debe tener en cuenta que esta desagregación puede afectar la columna noID y, por ende, los valores agregados para pagos y demás. </p>

In [ ]:
# # Aquí se pegan las columnas de noID que corresponden al sharepoint de EAPB (la base de datos fue descargada previamente en formato .xlsx)
# EPS = pd.read_excel('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Reg_EAPB.xlsx', dtype=str)
# # Se unen las dos bases de datos para obtener la información de FT004, FT003, FT005 y la base de datos noID en una sola base de datos
# FT004_FT003_FT005 = pd.merge(FT004_FT003_FT005, EPS, how='left', left_on=['nitproveedor'], right_on=['Nit'])
# # Se elimina el dataframe auxiliar para identificar las EPS por el sharepoint EAPB
# del [EPS]
# # Para conocer cuantas observaciones se tienen por tipo de aseguradora
# FT004_FT003_FT005.Reg_EAPB.value_counts()

<p> # Se carga la base de datos prestadores.xls para su posterior pega (se debe saber que esta base de datos debe ser modificada cada que se pueda identificando aquellos casos que aparecen como Objeto Social Diferente (OSD), pero en realidad pertenecen a otra categoria de prestador (IF, GIM, DM, TEP, Operador, entre otros). Es decir, en la base de datos prestadores figuran como OSD pero en realidad son IF por su naturaleza. </p>

In [17]:
# Se carga la base de datos prestadores.xls para su posterior pega (se debe saber que esta base de datos es modificada identificando aquellos casos que aparecen como Objeto Social Diferente (OSD), pero en realidad pertenecen a otra categoria de prestador (iF, GIM, DM, operador, entre otros). Es decir, en la base de datos prestadores figuran como OSD pero en realidad son IF por su naturaleza
prestadores = pd.read_excel('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/prestadores.xls', dtype=str)

In [18]:
# Se eliminan las columnas que no son de interés
prestadores = prestadores.drop(['clpr_codigo', 'naju_nombre'], axis=1)

In [19]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
FT004_FT003_FT005 = pd.merge(FT004_FT003_FT005, prestadores, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [20]:
# Se eilimina la base de datos auxiliar
del [prestadores]

In [21]:
# Se conocen las columnas del dataframe principal
FT004_FT003_FT005.columns

Index(['nit_eps', 'nitproveedor', 'ano', 'mes', 'CXP', 'DCXPmes', 'CXC',
       'DCXCmes', 'FT004_FT003', 'pagomes', 'causmes', 'giro', 'no_upc',
       'compra_cartera', 'pw', '_merge', 'nombre_prestador', 'clpr_nombre',
       'departamento', 'CONTROL'],
      dtype='object')

<p> La siguiente celda se debe ejecutar si se optó por utilizar la desagregación por aseguradora de la base de datos del sharepoint EAPB pues lo que hace es unir la clasificación que se tiene en dicha base con la de prestadores </p>

In [ ]:
# # La siguiente celda se debe ejecutar si se optó por utilizar la desagregación por aseguradora de la base de datos del sharepoint EAPB pues lo que hace es unir la clasificación que se tiene en dicha base con la de prestadores. Además, se llena los vacios con "nan" para facilitar el tratamiento posterior del tipo noID.
# noID = FT004_FT003_FT005['clpr_nombre'].astype(str).fillna('NA') + "-" + FT004_FT003_FT005['Reg_EAPB'].astype(str).fillna('NA')
# # Se crea la columna noID respecto a concatenar las columnas Reg_EAPB y clpr_nombre
# FT004_FT003_FT005['noID']=noID
# # Se libera espacio de la memoria
# del [noID]
# # Aquí se quitan los nan-
# FT004_FT003_FT005['noID'] = FT004_FT003_FT005['noID'].str.replace('nan-','')
# # Aquí se quitan los -nan
# FT004_FT003_FT005['noID'] = FT004_FT003_FT005['noID'].str.replace('-nan','')

In [22]:
# Se crea la columna noID respecto a la columna clpr_nombre tomada de la base de datos prestadores.xls
# RECORDAR QUE ESTA LINEA SE APLICA SOLO PARA LA AGREGACIÓN MAXIMA DE NOID (CUANDO NO SE TIENE EN CUENTA EL SHAREPOINT EAPB)
FT004_FT003_FT005['noID']=FT004_FT003_FT005['clpr_nombre'].astype(str)

In [24]:
# Se conocen el número de observaciones por cada noID
FT004_FT003_FT005.noID.value_counts()

nan                          16208105
IPS                            727371
IPS Pública                    492310
Profesional Independiente      239777
OSD                             87735
TEP                             15986
IF                               1506
DM                                536
Operador                          487
Name: noID, dtype: int64

In [ ]:
#FT004_FT003_FT005.loc[FT004_FT003_FT005['noID'].isnull(), 'noID'] = FT004_FT003_FT005['clpr_nombre']  # Para rellenar los vacios con "clpr_nombre"

In [25]:
# Se carga la base de datos operadores.dta para su posterior pega
operadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/operadores.dta')

In [26]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT004_FT003_FT005= pd.merge(FT004_FT003_FT005, operadores, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [27]:
# Para corregir el IPS cuando Tipo es vacío, por IPS-OL-DM-GIM
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == 'IPS') & 
       (FT004_FT003_FT005['tipo'].notnull()), 
       'noID'] = 'IPS-OL-DM-GIM'

In [28]:
# Se genera una nueva columna extrayendo el primer caracter del ID nitproveedor
FT004_FT003_FT005['d1'] = FT004_FT003_FT005['nitproveedor'].str[:1]

# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT004_FT003_FT005['largo'] = FT004_FT003_FT005.nitproveedor.str.len()

In [29]:
# Se limpia la clasificación por Tipo de ID
FT004_FT003_FT005['tipoid'] = 'NA' # Se crea la columna con categoria NA como base
# Condiciones
FT004_FT003_FT005.loc[
       (FT004_FT003_FT005['tipoid'] == 'NA') &
       (FT004_FT003_FT005['largo'] == 10) & # Para signar el tipo de acreedor CC
       (FT004_FT003_FT005['d1'] == '1'), 
       'tipoid'] = 'CC' 
FT004_FT003_FT005.loc[(FT004_FT003_FT005['tipoid'] == 'NA') &
       (FT004_FT003_FT005['largo'] == 9) & # Para signar el tipo de acreedor NI
       ((FT004_FT003_FT005['d1'] == '7') |
       (FT004_FT003_FT005['d1'] == '8') |
       (FT004_FT003_FT005['d1'] == '9')),  
       'tipoid'] = 'NI' 
FT004_FT003_FT005.loc[(FT004_FT003_FT005['tipoid'] == 'NA') &
       (FT004_FT003_FT005['tipoid'] != 'NI'), # Para completar el tipo de acreedor CC
       'tipoid'] = 'CC' 

<p> En las siguientes líneas se incluyen los noID que deben introducidos manualmente para generalizar la clasificación del proveedor </p>

In [30]:
# Se reemplazan los NIT de la nación
FT004_FT003_FT005.loc[(FT004_FT003_FT005['nitproveedor'] == "901037916") | 
       (FT004_FT003_FT005['nitproveedor'] == "900047282") |
       (FT004_FT003_FT005['nitproveedor'] == "899999014") |
       (FT004_FT003_FT005['nitproveedor'] == "900474727") |
       (FT004_FT003_FT005['nitproveedor'] == "900462447"),  
       'noID'] = 'Nacion'

In [31]:
# Se rellenan los noID que sean "nan" con el Tipo si el Tipo no es vacío
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == "nan") & 
       (FT004_FT003_FT005['tipo'].notnull()), 
       'noID'] = FT004_FT003_FT005['tipo']

In [32]:
# Para cambiar el noID por Persona Natural si en Tipo Proveedor se tiene CC y CE
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == 'nan') & 
       ((FT004_FT003_FT005['tipoid'] == 'CC') |
       (FT004_FT003_FT005['tipoid'] == 'CE')), 
       'noID'] = 'Persona Natural'

In [34]:
# Para cambiar el noID por Otro cuando es "nan"
FT004_FT003_FT005.loc[FT004_FT003_FT005['noID'] == 'nan', 'noID'] = 'Otro'

<p> Se reemplaza el noID de EPS por Otro. Este reemplazo se da ya que a veces las EPS se pagan a si mismas para cuadrar procesos administrativos y si se consideran estos pagos como a otro tercero se estaría incurriendo en problemas de contabilidad, por tal motivo, estos pagos se toman como pagos a Otro y no se incluye en las catergorias que se les dieron por separada gracias a la base de datos prestadores.xls </p>

In [35]:
# Se reemplaza el noID de EPS por Otro. Este reemplazo se da ya que a veces las EPS se pagan a si mismas para cuadrar procesos administrativos y si se consideran estos pagos como a otro tercero se estaría incurriendo en problemas de contabilidad, por tal motivo, estos pagos se toman como pagos a Otro y no se incluye en las catergorias que se les dieron por separada gracias a la base de datos prestadores.xls
FT004_FT003_FT005.loc[(FT004_FT003_FT005['nitproveedor'] == FT004_FT003_FT005['nit_eps']), 
       'noID'] = 'Otro'
# Para facilitar la agregación y la privacidad de las personas naturales, el NIT proveedor se reemplaza por los valores 123456789
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == 'Persona Natural'), 
       'nitproveedor'] = '123456789'

In [ ]:
FT004_FT003_FT005.noID.value_counts()

In [33]:
# Se crea un backup de la base en formato Feather
feather.write_feather(FT004_FT003_FT005, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT004_FT003_FT005_backup_feather_ene_may_2021') # Esta línea guarda la base de datos en binario 

# Aquí se recupera el back up
#FT004_FT003_FT005 = pd.read_feather('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT004_FT003_FT005_backup_feather_ene_may_2021')

<p>--------------------------</p>

In [36]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT004_FT003_FT005.rename(columns={'_merge': 'FT004_FT003_FT005'}, inplace=True)

In [37]:
# Se carga el archivo analisis para facilitar el tratamiento de la información por EPS
analisis = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/analisis.dta')

In [38]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
analisis.rename(columns={'nit': 'nit_eps'}, inplace=True)
# Se convierte la variable en str para facilitar su manipulación
analisis[['nit_eps', 'tipo']] = analisis[['nit_eps', 'tipo']].astype(str)

In [39]:
# Se unen las dos bases de datos para obtener la información de FT004, FT003, FT005 y la base de datos Analisis en una sola base de datos
FT004_FT003_FT005 = pd.merge(FT004_FT003_FT005, analisis, how='left', left_on=['nit_eps'], right_on=['nit_eps'])

In [40]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [analisis]

In [42]:
# Se encuentran las observaciones que cumplen las condiciones. Esta condición está basada en la base de datos analisis ya que se eliminan las observaciones que no aparecen en dicha base o nno tienen Tipo asignado en dicha base.
df_filtered = FT004_FT003_FT005[
    (FT004_FT003_FT005['tipo_y'].isnull())
    ].index

In [43]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004_FT003_FT005 = FT004_FT003_FT005.drop(df_filtered)

In [44]:
# Se especifica el tipo de variable que se tiene para facilitar el tratamiento de las variables
FT004_FT003_FT005[['DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes']] = FT004_FT003_FT005[['DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes']].astype(float)

In [45]:
# Se llenan con 0 los espacios vacíos de las variables de interés
FT004_FT003_FT005[['DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes']] = FT004_FT003_FT005[['DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes']].fillna(value=0)

<p> A partir de aquí se crea la columna PAGO. Es importante anotar que se utilizan diferentes objetos como auxiliares para optimizar el cálculo debido al volumen de la base de datos.</p>

In [46]:
# Se crea la columna PAGO tomando como referencia la columna pagomes
FT004_FT003_FT005['PAGO'] = FT004_FT003_FT005['pagomes'] 

In [47]:
# Se establecen las condiciones para establecer correctamente el PAGO
FT004_FT003_FT005.loc[(FT004_FT003_FT005['tipo_y'] == 'web'),
       'PAGO'] = FT004_FT003_FT005['pw']
       
FT004_FT003_FT005.loc[(FT004_FT003_FT005['tipo_y'] == 'giro'),
       'PAGO'] = FT004_FT003_FT005['giro']

In [48]:
# Objeto auxiliar
webgiro = FT004_FT003_FT005['pw'] + FT004_FT003_FT005['giro']

In [49]:
# Se crea la columna webgiro
FT004_FT003_FT005['webgiro'] = webgiro

In [50]:
# Objeto auxiliar
FT004_FT003_FT005.loc[(FT004_FT003_FT005['tipo_y'] == 'webgiro'),
       'PAGO'] = FT004_FT003_FT005['webgiro']

In [51]:
# Objeto auxiliar
SaldoCartera = FT004_FT003_FT005['CXP'] - FT004_FT003_FT005['CXC']

In [52]:
# Se crea la columna Saldo Cartera
FT004_FT003_FT005['SaldoCartera'] = SaldoCartera

In [53]:
# Objeto auxiliar
A = FT004_FT003_FT005['DCXPmes'] * (-1)

In [54]:
# Se genera una columna auxiliar para cambiar el signo de DCXPmes
FT004_FT003_FT005['A'] = A

In [55]:
# Se genera una columna auxiliar para cambiar el valor de DCXPmes
FT004_FT003_FT005['CALC'] = 0

In [56]:
# Objeto auxiliar
CALC = FT004_FT003_FT005['causmes'] + FT004_FT003_FT005['A'] + FT004_FT003_FT005['DCXCmes'] 

In [57]:
# Objeto auxiliar
FT004_FT003_FT005['CALC_AUX'] = CALC

In [58]:
# Objeto auxiliar
FT004_FT003_FT005.loc[(FT004_FT003_FT005['DCXCmes'] >= 0),  
       'CALC'] = FT004_FT003_FT005['CALC_AUX']

In [59]:
# Se genera una columna auxiliar para cambiar el valor de causmes
FT004_FT003_FT005['B'] = 0

In [60]:
# Objeto auxiliar
B = FT004_FT003_FT005['causmes'] + FT004_FT003_FT005['A']

In [61]:
# Objeto auxiliar
FT004_FT003_FT005['B_AUX'] = B

In [62]:
# Objeto auxiliar
FT004_FT003_FT005.loc[(FT004_FT003_FT005['DCXCmes'] < 0),  
       'B'] = FT004_FT003_FT005['B_AUX'] 

In [63]:
# Se reemplaza la columna CALC si se cumplen las condiciones
FT004_FT003_FT005.loc[(FT004_FT003_FT005['DCXCmes'] < 0),  
       'CALC'] = FT004_FT003_FT005['B']

In [64]:
# Se eliminan las columnas que no son de interés. Estas columnas fueron creadas como auxiliares. Estos objetos se crearon como auxiliares para los cálculos de interés
FT004_FT003_FT005 = FT004_FT003_FT005.drop(['A', 'B', 'var4'], axis=1)

In [65]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria. Estos objetos se crearon como auxiliares para los cálculos de interés
del [A, B, CALC, SaldoCartera, df_filtered, webgiro]

In [66]:
# Se carga la base de datos RUES.dta para su posterior pega
RUES = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/RUES_V2.dta')

In [67]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y RUES
FT004_FT003_FT005 = pd.merge(FT004_FT003_FT005, RUES, how='left', left_on=['nitproveedor'], right_on=['nitproveedor'])

In [68]:
# Se elimina la base de datos auxiliar de RUES
del [RUES]

In [69]:
# Para rellenar los Nombres de los prestadores vacios con Razón Social
FT004_FT003_FT005.loc[FT004_FT003_FT005['nombre_prestador'].notnull(), 'razónsocialonombre'] = FT004_FT003_FT005['nombre_prestador'] 

In [70]:
# Se eliminan las columnas que no son de interés
FT004_FT003_FT005 = FT004_FT003_FT005.drop(['nombre_prestador'], axis=1)

In [71]:
# Se rellenan los departamentos con Otro cuando es nulo
FT004_FT003_FT005.loc[(FT004_FT003_FT005['departamento'].isnull()), 
       'departamento'] = 'Otro'

<p> Desde aquí se hace el tratamiento para la agregación por EPS. Es decir, con las siguientes líneas se crea la base de datos agregada por EPS. </p>

In [84]:
# Aqui se guarda la base de datos agregada por EPS para las variables de interés 
FT004_FT003_FT005_EPS = FT004_FT003_FT005.groupby(['nit_eps', 'ano', 'mes', 'eps', 'tipo_y']).agg({'CXP':'sum', 'DCXPmes':'sum', 'CXC':'sum', 'DCXCmes':'sum', 'pw':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'causmes':'sum', 'pagomes':'sum', 'PAGO':'sum', 'CALC':'sum','pagomes':'sum'}).reset_index()

In [85]:
# Se carga el archivo tratado del FT006 que fue grabado como causaprestador
FT006 = pd.read_csv('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/DVFICEPS_2020_ene_may_2021.csv', encoding='utf-8-sig', dtype={'nit_eps':object, 'ano': object, 'mes': object})

In [86]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
# FT006.rename(columns={'ï»¿nit_eps': 'nit_eps'}, inplace=True)
# Se convierte la variable NIT EPS como str
FT006[['nit_eps']] = FT006[['nit_eps']].astype(str)

In [87]:
# Se unen las dos bases de datos para obtener la información de FT004, FT003, FT005 y FT006 en una sola base de datos
FT004_FT003_FT005_FT006 = pd.merge(FT004_FT003_FT005_EPS, FT006, how='outer', left_on=['nit_eps', 'ano', 'mes'], right_on=['nit_eps', 'ano', 'mes'])

In [88]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT006, FT004_FT003_FT005_EPS, operadores]

NameError: name 'operadores' is not defined

In [89]:
# Se carga el archivo tratado del FT007 que fue grabado como causaprestador
FT007 = pd.read_csv('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/compratitulos_2020_ene_may_2021.csv', encoding='utf-8-sig', dtype={'nit_eps':object, 'ano': object, 'mes': object})

In [90]:
# Eliminamos los años que no nos interesa del FT007
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT007[
    (FT007['ano'] != '2020') &
    (FT007['ano'] != '2021')
    ].index

In [91]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT007 = FT007.drop(df_filtered)

In [92]:
# Se convierte la variable en mes y año
FT007[['ano', 'mes']] = FT007[['ano', 'mes']].astype(float).astype(int).astype(str)

In [93]:
# Se unen las dos bases de datos para obtener la información de FT004, FT003, FT005, FT006 y FT007 en una sola base de datos
FT004_FT003_FT005_FT006_FT007 = pd.merge(FT004_FT003_FT005_FT006, FT007, how='outer', left_on=['nit_eps', 'ano', 'mes'], right_on=['nit_eps', 'ano', 'mes'])

In [94]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004_FT003_FT005_FT006, FT007, df_filtered]

In [95]:
# Se carga el archivo tratado del FT007 que fue grabado como causaprestador
FT001 = pd.read_csv('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/ingresosyefectivo_2020_2021_V2.csv', encoding='utf-8-sig', dtype={'nit_eps':object, 'ano': object, 'mes': object})

In [96]:
# Se unen las dos bases de datos para obtener la información de FT004, FT003, FT005, FT006 y FT007 en una sola base de datos
FT00143567 = pd.merge(FT004_FT003_FT005_FT006_FT007, FT001, how='outer', left_on=['nit_eps', 'ano', 'mes'], right_on=['nit_eps', 'ano', 'mes'])

In [97]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT004_FT003_FT005_FT006_FT007, FT001]

In [98]:
# Se convierten a float las varaibles relevantes
FT00143567[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC', 'DSaldo', 'comprames', 'DCMCOPmes', 'DEfectivo', 'EnfH', 'NA', 'DPMAXmes', 'DUPCmes', 'valorEfectivo']] = FT00143567[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC', 'DSaldo', 'comprames', 'DCMCOPmes', 'DEfectivo', 'EnfH', 'NA', 'DPMAXmes', 'DUPCmes', 'valorEfectivo']].astype(float)

In [99]:
# Se llenan con 0 los espacios vacíos de las variables de interés
FT00143567[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC', 'DSaldo', 'comprames', 'DCMCOPmes', 'DEfectivo', 'EnfH', 'NA', 'DPMAXmes', 'DUPCmes', 'valorEfectivo']] = FT00143567[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC', 'DSaldo', 'comprames', 'DCMCOPmes', 'DEfectivo', 'EnfH', 'NA', 'DPMAXmes', 'DUPCmes', 'valorEfectivo']].fillna(value=0)

In [100]:
# Auxiliar
inversiones = FT00143567['DSaldo'] + FT00143567['comprames']

In [101]:
# Se crea la columna inversiones sumando DSaldomes y comprames
FT00143567['inversiones'] = inversiones

In [102]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [inversiones]

In [103]:
# Aqui se guarda la base de datos agregada por ingresos y pagos de las EPS (Corresponde a la línea 692 del código en stata que guarda la información en la hoja EPS)
FT00143567_EPS = FT00143567.groupby(['nit_eps', 'mes', 'ano', 'eps', 'tipo_y']).agg({'CXP':'sum', 'DCXPmes':'sum', 'CXC':'sum', 'DCXCmes':'sum', 'pw':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'causmes':'sum', 'pagomes':'sum', 'PAGO':'sum', 'CALC':'sum', 'DSaldo':'sum', 'comprames':'sum', 'inversiones':'sum', 'DCMCOPmes':'sum','DEfectivo':'sum','EnfH':'sum','NA':'sum','DPMAXmes':'sum','DUPCmes':'sum', 'valorEfectivo':'sum'}).reset_index()

In [106]:
# Se convierte la variable en mes y año
FT00143567_EPS[['ano', 'mes']] = FT00143567_EPS[['ano', 'mes']].astype(int)

In [107]:
# Se encuentran las observaciones que cumplen las condiciones para no tener en cuenta observaciones posteriores a las de los meses de interés
df_filtered = FT00143567_EPS[
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 6) |
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 7) |
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 8) |
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 9) |
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 10) |
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 11) |
    (FT00143567_EPS['ano'] == 2021) & (FT00143567_EPS['mes'] == 12)
    ].index

In [108]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT00143567_EPS = FT00143567_EPS.drop(df_filtered)

In [128]:
# Se encuentran las observaciones que cumplen las condiciones para no tener en cuenta observaciones posteriores a las de los meses de interés
df_filtered = FT00143567_EPS[
    (FT00143567_EPS['ano'] == 2019)
    ].index
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT00143567_EPS = FT00143567_EPS.drop(df_filtered)

In [129]:
FT00143567_EPS.ano.value_counts()

2020    444
2021    183
Name: ano, dtype: int64

In [130]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT00143567_EPS.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/EPS_analisis_2020_enemay2021_V1.xlsx', index = False)
# FT00143567_EPS.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/EPS_analisis_2020_2021.csv', index = False, encoding='utf-8-sig')
# feather.write_feather(FT00143567_EPS, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/EPS_analisis_2020_2021.feather') # Esta línea guarda la base de datos en binario para optimizar el tiempo de cargay el espacio

In [131]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT00143567_EPS.to_stata(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/EPS_analisis_2020_enemay2021_V1.dta')

#<p> Aquí se trata la base de datos Terceros. Es decir, con las siguientes líneas se crea la base de datos agregada por Tercero. </p>

In [111]:
# Se convierten a float las varaibles relevantes
FT004_FT003_FT005[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC']] = FT004_FT003_FT005[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC']].astype(float)

In [112]:
# Se llenan con 0 los espacios vacíos de las variables de interés
FT004_FT003_FT005[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC']] = FT004_FT003_FT005[['PAGO', 'DCXPmes', 'DCXCmes', 'pw', 'giro', 'no_upc', 'compra_cartera', 'CXP', 'CXC', 'causmes', 'pagomes', 'CALC']].fillna(value=0)

In [113]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT004_FT003_FT005 = FT004_FT003_FT005[
    (FT004_FT003_FT005['PAGO'] != 0)|
    (FT004_FT003_FT005['DCXPmes'] != 0)|
    (FT004_FT003_FT005['DCXCmes'] != 0)|
    (FT004_FT003_FT005['pw'] != 0)|
    (FT004_FT003_FT005['giro'] != 0)|
    (FT004_FT003_FT005['no_upc'] != 0)|
    (FT004_FT003_FT005['compra_cartera'] != 0)
    ]

In [114]:
# Variable auxiliar
A = FT004_FT003_FT005['noID'] + '-' + FT004_FT003_FT005['DESC_SECCIÓN']

In [115]:
# Se crea la columna auxiliar para modificar el noID
FT004_FT003_FT005['A'] = A

<ipython-input-115-b2a38305e7b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FT004_FT003_FT005['A'] = A


In [116]:
# Se reemplaza el noID si cumple las condiciones
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == 'Otro') &
       (FT004_FT003_FT005['DESC_SECCIÓN'].notnull()), 
       'noID'] = FT004_FT003_FT005['A']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [117]:
# Se reemplaza el noID si cumple las condiciones
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == FT004_FT003_FT005['A']) |
       (FT004_FT003_FT005['noID'] == 'Otro'), 
       'nitproveedor'] = "Otros proveedores"

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [118]:
# Se reemplaza el noID si cumple las condiciones
FT004_FT003_FT005.loc[(FT004_FT003_FT005['noID'] == FT004_FT003_FT005['A']) |
       (FT004_FT003_FT005['noID'] == 'Otro'), 
       'razónsocialonombre'] = FT004_FT003_FT005['noID']

In [119]:
# Se transforman los datos para su correcta identificación
FT004_FT003_FT005[['ano', 'mes']] = FT004_FT003_FT005[['ano', 'mes']].astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [120]:
# Se encuentran las observaciones que cumplen las condiciones para no tener en cuenta observaciones posteriores a las de los meses de interés
df_filtered = FT004_FT003_FT005[
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 5) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 6) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 7) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 8) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 9) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 10) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 11) |
    (FT004_FT003_FT005['ano'] == 2021) & (FT004_FT003_FT005['mes'] == 12)
    ].index

In [121]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004_FT003_FT005 = FT004_FT003_FT005.drop(df_filtered)

In [122]:
# Se transforman los datos para su correcta identificación
FT004_FT003_FT005[['nit_eps', 'nitproveedor', 'noID', 'departamento', 'eps', 'tipo_y', 'razónsocialonombre']] = FT004_FT003_FT005[['nit_eps', 'nitproveedor', 'noID', 'departamento', 'eps', 'tipo_y', 'razónsocialonombre']].astype(str)

In [123]:
# Se hace la agrupación requerida por Tercero sumando los valores de las variables de interés
FT004_FT003_FT005_tercero = FT004_FT003_FT005.groupby(['nit_eps', 'nitproveedor', 'noID', 'departamento', 'eps', 'tipo_y', 'razónsocialonombre', 'mes', 'ano']).agg({'CXP':'sum', 'DCXPmes':'sum', 'CXC':'sum', 'DCXCmes':'sum', 'pw':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'causmes':'sum', 'pagomes':'sum', 'PAGO':'sum', 'CALC':'sum'}).reset_index()

In [124]:
# Se transforman los datos para su correcta identificación
FT004_FT003_FT005_tercero[['nit_eps', 'nitproveedor', 'noID', 'departamento', 'eps', 'tipo_y', 'razónsocialonombre']] = FT004_FT003_FT005_tercero[['nit_eps', 'nitproveedor', 'noID', 'departamento', 'eps', 'tipo_y', 'razónsocialonombre']].astype(str)

In [132]:
# Se encuentran las observaciones que cumplen las condiciones para no tener en cuenta observaciones posteriores a las de los meses de interés
df_filtered = FT004_FT003_FT005_tercero[
    (FT004_FT003_FT005_tercero['ano'] == 2019)
    ].index
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT004_FT003_FT005_tercero = FT004_FT003_FT005_tercero.drop(df_filtered)

In [134]:
# Se crean los valores previos para asignar categorias
difSNSWeb_control = (FT004_FT003_FT005_tercero['PAGO'] - FT004_FT003_FT005_tercero['pw'])
difSNSadres_control = (FT004_FT003_FT005_tercero['PAGO'] - FT004_FT003_FT005_tercero['giro'] - FT004_FT003_FT005_tercero['no_upc'] - FT004_FT003_FT005_tercero['compra_cartera'])
difSNScalc_control = (FT004_FT003_FT005_tercero['PAGO'] - FT004_FT003_FT005_tercero['CALC'])

In [135]:
# Se crean columns de control para signar categorias
# Se crea la columna noID tomando como referencia la columna clpr_nombre
FT004_FT003_FT005_tercero['difSNSWeb_control'] = difSNSWeb_control
FT004_FT003_FT005_tercero['difSNSadres_control'] = difSNSadres_control
FT004_FT003_FT005_tercero['difSNScalc_control'] = difSNScalc_control

In [136]:
# Se limpia la clasificación por diferencias entre Pago, pw, giro y Calc
FT004_FT003_FT005_tercero['difSNSWeb'] = 'Igual'
FT004_FT003_FT005_tercero['difSNSadres'] = 'Igual'
FT004_FT003_FT005_tercero['difSNScalc'] = 'Igual'
# Condiciones
FT004_FT003_FT005_tercero.loc[(FT004_FT003_FT005_tercero['difSNSWeb_control'] > 0), 
       'difSNSWeb'] = 'Mayor' 

FT004_FT003_FT005_tercero.loc[(FT004_FT003_FT005_tercero['difSNSWeb_control'] < 0), 
       'difSNSWeb'] = 'Menor' 


FT004_FT003_FT005_tercero.loc[(FT004_FT003_FT005_tercero['difSNSadres_control'] > 0), 
       'difSNSWeb'] = 'Mayor' 

FT004_FT003_FT005_tercero.loc[(FT004_FT003_FT005_tercero['difSNSadres_control'] < 0), 
       'difSNSWeb'] = 'Menor' 


FT004_FT003_FT005_tercero.loc[(FT004_FT003_FT005_tercero['difSNScalc_control'] > 0), 
       'difSNSWeb'] = 'Mayor' 

FT004_FT003_FT005_tercero.loc[(FT004_FT003_FT005_tercero['difSNScalc_control'] < 0), 
       'difSNSWeb'] = 'Menor' 

In [137]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT004_FT003_FT005_tercero.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Tercero_analisis_2020_enemay2021_V1.xlsx', index = False)
# FT004_FT003_FT005_tercero.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Tercero_analisis_2020_2021.csv', index = False, encoding='utf-8-sig')
# feather.write_feather(FT004_FT003_FT005_tercero, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Tercero_analisis_2020_2021.feather') # Esta línea guarda la base de datos en binario para optimizar el tiempo de cargay el espacio

In [138]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT004_FT003_FT005_tercero.to_stata(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Tercero_analisis_2020_enemay2021_V1.dta')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\stata.py:2398: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    razónsocialonombre   ->   raz_nsocialonombre

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)


In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [A, FT00143567, FT00143567_EPS, FT004_FT003_FT005 , FT004_FT003_FT005_EPS, FT004_FT003_FT005_tercero, df_filtered, difSNSWeb_control, difSNSadres_control ,difSNScalc_control]